# Basic Message - Alice

**Should be run alongside [Bob](http://localhost:8889/notebooks/Part%207%20-%20Basic%20Message.ipynb)**

In [1]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# WARNING: You should use environment variables for this
# TODO: Make env variables accessible through juypter notebooks
API_KEY = "alice_api_123456789"


# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, api_key=API_KEY)

## Setup a listener for the basicmessages topic

This is emitted using PyPubSub when the controller receives a basicmessages webhook from the agent. This happens everytime the agent receives a basicmessage.

In [3]:
def messages_handler(payload):
    connection_id = payload["connection_id"]
    print("Handle message", payload, connection_id)


message_listener = {
    "handler": messages_handler,
    "topic": "basicmessages"
}

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

agent_controller.register_listeners([message_listener], defaults=True)

## Verify an Active Connection Exists

This should have been established through the setup docker container using the create_connection.py script.

In [3]:
response = await agent_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
    else:
        print("Connection is still progressing to active state, retry in a few moments")
else:
    print("You must create a connection")
    

Results :  [{'updated_at': '2020-09-08 10:24:53.211339Z', 'my_did': '2PV1LoGz2pZHTvyZ85y6vr', 'connection_id': 'd88d311a-c35a-4eb1-9d42-3b49ef79124d', 'created_at': '2020-09-08 10:24:41.122406Z', 'state': 'active', 'routing_state': 'none', 'invitation_mode': 'once', 'their_label': 'Bob', 'their_did': 'MZhiv2XJb99Sibjw6sYNqv', 'invitation_key': 'EWEPikZ15cU7NQwRusVN6665auSK8xFBoQarmAUPTb5x', 'request_id': '534482d3-abe4-4fc1-b193-3e3738ecc36f', 'initiator': 'external', 'accept': 'manual'}]
Connection : {'updated_at': '2020-09-08 10:24:53.211339Z', 'my_did': '2PV1LoGz2pZHTvyZ85y6vr', 'connection_id': 'd88d311a-c35a-4eb1-9d42-3b49ef79124d', 'created_at': '2020-09-08 10:24:41.122406Z', 'state': 'active', 'routing_state': 'none', 'invitation_mode': 'once', 'their_label': 'Bob', 'their_did': 'MZhiv2XJb99Sibjw6sYNqv', 'invitation_key': 'EWEPikZ15cU7NQwRusVN6665auSK8xFBoQarmAUPTb5x', 'request_id': '534482d3-abe4-4fc1-b193-3e3738ecc36f', 'initiator': 'external', 'accept': 'manual'}
Active Conne

## Send a Basic Message over DIDComm to Bob

See [aries-rfc](https://github.com/hyperledger/aries-rfcs/tree/master/features/0095-basic-message)

You can send as messages as you want, if you are running the [basic-message tutorial on Bob's notebook](http://localhost:8889/notebooks/Part%207%20-%20Basic%20Message.ipynb) these will be received and printed by the message handler. You may have to run a code block to see the output.

In [5]:
basic_message = "hello from Alice"
response = await agent_controller.messaging.send_message(connection_id, basic_message)
print("BASIC MESSAGE - Alice -> Bob")
print(response)

BASIC MESSAGE - Alice -> Bob
{}
Handle message {'connection_id': '23ebe2a6-33f5-4f60-a2bd-dbabdd885437', 'message_id': '1cfd05f4-2a82-409e-912e-0d0d5addb754', 'content': 'This is a response from Bob', 'state': 'received'} 23ebe2a6-33f5-4f60-a2bd-dbabdd885437


## End of Tutorial

Be sure to terminate the controller so you can run another tutorial.

In [4]:
response = await agent_controller.terminate()
print(response)

None
